PILAR ARIAS, YEDRA MARTÍN, PATRICIA MEDIAVILLA, AMAYA SARASA

<GRUPO 9>

#Práctica 1

##Introducción

Esta es la primera práctica del segundo laboratorio. Dada una base de datos en formato .csv, se nos pide contestar a una serie de cuestiones. Para ello, lo priemro que debemos hacer es cargar y leer la base de datos:

In [4]:
#Importar librerías
#!apt install mlxtend
import numpy as np #numpy
import pandas as pd #pandas
import sklearn as sk #scikit-learn
import matplotlib.pyplot as plt #gráfica Matplotlib
#from mlxtend.frequent_patterns import apriori
#from apyori import apriori #apriori
from mlxtend.preprocessing import TransactionEncoder
#from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori, association_rules
#from mlxtend.frequent_patterns import frequent_patterns
#from mlxtend import frequent_patterns
#from mlxtend.frequent_patterns import frequent_patterns

#Importamos la base de datos
df = pd.read_csv('BlackFriday.csv',sep=',')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233599 entries, 0 to 233598
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     233599 non-null  int64  
 1   Product_ID                  233599 non-null  object 
 2   Gender                      233599 non-null  object 
 3   Age                         233599 non-null  object 
 4   Occupation                  233599 non-null  int64  
 5   City_Category               233599 non-null  object 
 6   Stay_In_Current_City_Years  233599 non-null  object 
 7   Marital_Status              233599 non-null  int64  
 8   Product_Category_1          233599 non-null  int64  
 9   Product_Category_2          161255 non-null  float64
 10  Product_Category_3          71037 non-null   float64
dtypes: float64(2), int64(4), object(5)
memory usage: 19.6+ MB


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


----------------

----------------

Subimos la base de datos, df.info nos aporta la infomación que contiene la base de datos, vemos el tipo, la cantidad de las variables con las que trabajamos... y .head vamos a ver que forma tiene la base de datos y las primeras filas de esta.

Según se ha estudiado en clase, encontramos dos métodos para afrontarlo. Nos decantaremos por CRISP-DM. Las siglas para: CRoss-Industry Standard Process for Data Mining. 

Es un modelo de proceso de minería de datos que se utiliza en la industria. Es un marco de trabajo que define un proceso estandarizado para llevar a cabo proyectos de minería de datos, desde la definición del problema hasta la implementación y evaluación del resultado. Los seis pasos que componen Crisp-DM son los siguientes:

- Comprensión de los negocios y definición del problema: comprender la naturaleza del problema y los objetivos de negocio.

- Comprensión de los datos: recopilación inicial, descripción y exploración  de datos.

- Preparación de los datos: Selección, limpieza, construcción, integración, formateo de los datos.

- Modelado: aplicar las técnicas de minería de datos a los dataset.

- Evaluación: determinar si los resultados son útiles a las necesidades del negocio.

- Despliegue: explotar utilidad de los modelos, integrándolos en las tareas de toma de decisiones de la organización -> Call to Action. 

##Comprensión de los negocios y definición del problema

La librería se usará de la siguiente manera:
1) Empezaremos obteniendo los itemsets frecuentes para k=1. En este punto necesitaremos obtener el
soporte de los itemset. Por lo tanto, se tendrá que usar un método que dado un itemset devuelva su
soporte.

2) Para k≥2.
a. Mostrar los itemset frecuentes candidatos y su soporte.

3) Repetir el proceso 2 hasta que no se generen nuevos itemsets frecuentes.

4) Mostrar todas las posibles reglas con la confianza de cada una de ellas.

5) Listar todas las reglas que sean de alta confianza.

6) Usar los siguientes métodos. Dado un antecesor, devolver todas las reglas que contengan a dicho
antecesor. Dado un umbral mínimo devolver todas las reglas que cumplan con dicha confianza.

7) Utiliza las representaciones gráficas que consideres adecuadas para representar las reglas obtenidas
y obtener conclusiones a partir de los datos.

A partir de lo anterior de pide:
- Prueba al menos tres configuraciones de soporte y frecuencia para cada género, edad y tipo de producto.
- ¿Qué diferencias hay entre usar soporte y frecuencia? Respalda la respuesta con datos 
- ¿Qué tipo reglas desaparecen según la configuración y categorías (género, edad y tipo de producto)
consideradas? ¿Por qué? 
- Para una de las configuraciones, interpreta algunas de las reglas que te hayan resultado interesantes
obtenidas usando la clase association_rules y un par de configuraciones cambiando la variable metric y
min_threshold. Justifica los resultados. 
- Dadas las mejores configuraciones. ¿Existen reglas o patrones que se repitan? ¿Podemos generalizar de
alguna manera como se comportan los clientes? Expón las conclusiones respaldadas con datos 

## Comprensión de los datos

La base de datos que utilizaremos consta de 233599 observaciones. Teniendo en ceunta que el trabajo que vamos a realizar, utilizanod el algoritmo apriori, queremos sacar regalas y con todos estos datos, lo que nos hace pensar es que vamos a encontrar muchísimas reglas a partir de estos datos.

La clave está en cuales son las que más nos interesan, basandonos en su confianza, y una vez hallado esto debemos tomar decisiones para que usando estas reglas vendamos el mayor número de producto, para aumentar nuestras gananzas.

Existen 11 tipos de atributos distintos: 

- User_ID: número para identificar al usuario.
- Product_ID: número para identificar el producto.
- Gender: Género del comprador.	
- Age: Edad del comprador.
- Occupation: A lo que se dedica el comprador, ya pasasdo a categórica.
- City_Category: Categoría de la ciudad donde reside el user.
- Stay_In_Current_City_Years: El tiempo que lleva en la ciudad.
- Marital_Status: Estado civil.
- Product_Category_1: La categoría principal del producto.
- Product_Category_2: Siguente categoría.
- Product_Category_3: Última categoría del producto.



##Preparación de los datos

In [5]:
usernan = df['User_ID'].isna().sum()
print(f"La cantidad de NaN que existen en la variable User_ID es:{usernan}")
productnan = df['Product_ID'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_ID es:{productnan}")
gendernan = df['Gender'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Gender es:{gendernan}")
agenan = df['Age'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Age es:{agenan}")
ocupationan = df['Occupation'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Occupation es:{ocupationan}")
City_Categorynan = df['City_Category'].isna().sum()
print(f"La cantidad de NaN que existen en la variable City_Category es:{City_Categorynan}")
Stay_In_Current_City_Yearsnan = df['Stay_In_Current_City_Years'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Stay_In_Current_City_Years es:{Stay_In_Current_City_Yearsnan}")
Marital_Statusnan = df['Marital_Status'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Marital_Status es:{Marital_Statusnan}")
Product_Category_1nan = df['Product_Category_1'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_1 es:{Product_Category_1nan}")
Product_Category_2nan = df['Product_Category_2'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_2 es:{Product_Category_2nan}")
Product_Category_3nan = df['Product_Category_3'].isna().sum()
print(f"La cantidad de NaN que existen en la variable Product_Category_3 es:{Product_Category_3nan}")

La cantidad de NaN que existen en la variable User_ID es:0
La cantidad de NaN que existen en la variable Product_ID es:0
La cantidad de NaN que existen en la variable Gender es:0
La cantidad de NaN que existen en la variable Age es:0
La cantidad de NaN que existen en la variable Occupation es:0
La cantidad de NaN que existen en la variable City_Category es:0
La cantidad de NaN que existen en la variable Stay_In_Current_City_Years es:0
La cantidad de NaN que existen en la variable Marital_Status es:0
La cantidad de NaN que existen en la variable Product_Category_1 es:0
La cantidad de NaN que existen en la variable Product_Category_2 es:72344
La cantidad de NaN que existen en la variable Product_Category_3 es:162562


##Modelado

In [6]:
df_User_Product = (df
        .groupby(['User_ID', 'Product_ID'])['Product_Category_1']
        .sum()
        .apply(lambda x:x>0)
        .unstack()
        .fillna(False)
        .reset_index()
        .set_index('User_ID')
          )
df_User_Product

Product_ID,P00000142,P00000242,P00000342,P00000442,P00000542,P00000642,P00000742,P00000842,P00000942,P00001042,...,P0099042,P0099142,P0099242,P0099342,P0099442,P0099542,P0099642,P0099742,P0099842,P0099942
User_ID,,,,,,,,,,,,,,,,,,,,,
1000001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000002,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000003,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000004,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000005,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006036,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False
1006037,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1006038,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
df_Gender = (df
            .groupby(['User_ID','Gender'])['Product_Category_1']
            .sum()
            .apply(lambda x: x>0)
            .unstack()
            .fillna(False)
            .reset_index()
            .set_index('User_ID')
)

In [8]:
df_Age = (df
        .groupby(['User_ID','Age'])['Product_Category_1']
        .sum()
        .apply(lambda x: x>0)
        .unstack()
        .fillna(False)
        .reset_index()
        .set_index('User_ID')
)

In [9]:
df_Occupation = (df
                .groupby(['User_ID','Occupation'])['Product_Category_1']
                .sum()
                .apply(lambda x: x>0)
                .unstack()
                .fillna(False)
                .reset_index()
                .set_index('User_ID')
)

In [10]:
df_City = (df
          .groupby(['User_ID','City_Category'])['Product_Category_1']
          .sum()
          .apply(lambda x: x>0)
          .unstack()
          .fillna(False)
          .reset_index()
          .set_index('User_ID')
)

In [11]:
df_Stay = (df
           .groupby(['User_ID','Stay_In_Current_City_Years'])['Product_Category_1']
           .sum()
           .apply(lambda x: x>0)
           .unstack()
           .fillna(False)
           .reset_index()
           .set_index('User_ID')
)

In [12]:
df_Marital = (df
              .groupby(['User_ID','Marital_Status'])['Product_Category_1']
              .sum()
              .apply(lambda x: x>0)
              .unstack()
              .fillna(False)
              .reset_index()
              .set_index('User_ID')
)

In [13]:
df_Product_Category_1 = (df
                         .groupby(['User_ID','Product_Category_1'])['Product_Category_2']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [14]:
df_Product_Category_2 = (df
                         .groupby(['User_ID','Product_Category_2'])['Product_Category_1']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [15]:
df_Product_Category_3 = (df
                         .groupby(['User_ID','Product_Category_3'])['Product_Category_1']
                         .sum()
                         .apply(lambda x: x>0)
                         .unstack()
                         .fillna(False)
                         .reset_index()
                         .set_index('User_ID')
)

In [16]:
data = pd.concat([df_User_Product, df_Occupation, df_City, df_Stay, df_Marital, df_Product_Category_1, df_Product_Category_2, df_Product_Category_3, df_Age, df_Gender], axis = 1)
data

,P00000142,P00000242,P00000342,P00000442,P00000542,P00000642,P00000742,P00000842,P00000942,P00001042,...,18.0,0-17,18-25,26-35,36-45,46-50,51-55,55+,F,M
User_ID,,,,,,,,,,,,,,,,,,,,,
1000001,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
1000002,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
1000003,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,True
1000004,False,False,False,False,False,False,False,False,False,False,...,NaN,False,False,False,False,True,False,False,False,True
1000005,False,False,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006036,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,True,False
1006037,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
1006038,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False


In [20]:
print(data.shape)

(5891, 3581)


In [23]:
#Esto es para k=1 // use_colnames = True, max_len = 3
#frq_items = apriori(df, min_support = 0.05, use_colnames = True, max_len = 3)
data.fillna(0)
frequent_itemsets = apriori(data, min_support=0.05, use_colnames=True, max_len=3)
frequent_itemsets

ValueError: ignored

In [ ]:
type(frequent_itemsets)

pandas.core.frame.DataFrame

In [ ]:
#Para k=2

frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets2=frequent_itemsets[frequent_itemsets['length'] == 2]
frequent_itemsets2.head()

,support,itemsets,length
1200,0.016466,"(P00025442, P00000142)",2
1201,0.010185,"(P00028842, P00000142)",2
1202,0.012052,"(P00031042, P00000142)",2
1203,0.011713,"(P00000142, P00034742)",2
1204,0.010185,"(P00000142, P00044442)",2


In [ ]:
#Para k=3
frequent_itemsets3=frequent_itemsets[frequent_itemsets['length'] == 3]
frequent_itemsets3

,support,itemsets,length


COPIO CÓDIGO QUE ES INTERESANTE PARA FUTURO


In [ ]:
#frequent_itemsets = apriori(data, min_support=0.6, use_colnames=True)
#frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
#frequent_itemsets1=frequent_itemsets[frequent_itemsets['length'] == 1]

In [ ]:
#frequent_itemsets[ (frequent_itemsets['length'] >1 ) & #esto sería para k=2
                   #(frequent_itemsets['support'] >= 0.8) ]

,support,itemsets,length


Para sacar las reglas de asocuación

In [ ]:
#Reglas de asociación
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(P00025442),(P00000142),0.117977,0.082159,0.016466,0.139568,1.698754,0.006773,1.066721
1,(P00000142),(P00025442),0.082159,0.117977,0.016466,0.200413,1.698754,0.006773,1.103099
2,(P00028842),(P00000142),0.087591,0.082159,0.010185,0.116279,1.415289,0.002989,1.038609
3,(P00000142),(P00028842),0.082159,0.087591,0.010185,0.123967,1.415289,0.002989,1.041523
4,(P00031042),(P00000142),0.091156,0.082159,0.012052,0.132216,1.609266,0.004563,1.057683
...,...,...,...,...,...,...,...,...,...
2078,(P00303042),(P00265242),0.047021,0.140723,0.010864,0.231047,1.641855,0.004247,1.117463
2079,(P00324942),(P00265242),0.051944,0.140723,0.011883,0.228758,1.625590,0.004573,1.114147
2080,(P00334242),(P00265242),0.071465,0.140723,0.010864,0.152019,1.080270,0.000807,1.013321
2081,(P0097242),(P00277642),0.066542,0.071125,0.010015,0.150510,2.116123,0.005282,1.093450


In [ ]:
rulesaltaconfianza = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.25)
rulesaltaconfianza

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(P00030842),(P00025442),0.045323,0.117977,0.011373,0.250936,2.127001,0.006026,1.177501
1,(P00127842),(P00025442),0.046002,0.117977,0.011543,0.250923,2.126884,0.006116,1.177480
2,(P00275842),(P00025442),0.038024,0.117977,0.010185,0.267857,2.270427,0.005699,1.204715
3,(P00030842),(P00237542),0.045323,0.102699,0.011543,0.254682,2.479884,0.006888,1.203916
4,(P00070042),(P00046742),0.049567,0.109659,0.012392,0.250000,2.279799,0.006956,1.187122
5,(P00127442),(P00046742),0.041080,0.109659,0.010864,0.264463,2.411688,0.006359,1.210464
6,(P00120042),(P00110942),0.065184,0.104566,0.018503,0.283854,2.714586,0.011687,1.250351
7,(P0097242),(P00110942),0.066542,0.104566,0.017484,0.262755,2.512809,0.010526,1.214568
8,(P00119742),(P00112142),0.046851,0.121711,0.011883,0.253623,2.083813,0.006180,1.176737
9,(P00125942),(P00112142),0.052623,0.121711,0.013410,0.254839,2.093800,0.007006,1.178656


##Evaluación

##Despliegue